# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import randint
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, classification_report

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [13]:
data = pd.read_csv("Dataset_inisiasi.csv")
data.head()

,TransactionAmount,TransactionDuration,CustomerAge,AccountBalance,Cluster,TransactionType,Channel
0,14.09,81.0,70.0,5112.21,4,Debit,ATM
1,376.24,141.0,68.0,13758.91,4,Debit,ATM
2,126.29,56.0,19.0,1122.35,1,Debit,Online
3,184.50,25.0,26.0,8569.06,1,Debit,Online
4,13.45,198.0,26.0,7429.40,3,Credit,Online


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [14]:
le = LabelEncoder()
data[['TransactionType', 'Channel']] = data[['TransactionType', 'Channel']].apply(le.fit_transform)

X = data.drop(columns=['Cluster'])
y = data['Cluster']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [15]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
y_pred_dt = dt_model.predict(X_test)

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

# Penjelasan Algoritma yang digunakan
- Random Forrest merupakan ensemble learning yang menggabungkan beberapa Decision Tree untuk meningkatkan akurasi prediksi dan mengurangi risiko overfitting. Setiap pohon dalam Random Forest dilatih menggunakan subset acak dari data pelatihan dan subset acak dari fitur yang tersedia.
- Decision Tree merupakan algoritma machine learning yang sering digunakan dalam tugas klasifikasi dan regresi. Struktur dasar dari Decision Tree melibatkan tiga komponen utama, yaitu akar (root node), node (decision node), dan daun (leaf node)

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [16]:
print("Random Forest Model")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Model
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00       113
           2       1.00      1.00      1.00       124
           3       1.00      1.00      1.00       114
           4       1.00      1.00      1.00       152

    accuracy                           1.00       503
   macro avg       1.00      1.00      1.00       503
weighted avg       1.00      1.00      1.00       503



In [17]:
print("Decision Tree Model")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Classification Report:\n", classification_report(y_test, y_pred_dt))

Decision Tree Model
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00       113
           2       1.00      1.00      1.00       124
           3       1.00      1.00      1.00       114
           4       1.00      1.00      1.00       152

    accuracy                           1.00       503
   macro avg       1.00      1.00      1.00       503
weighted avg       1.00      1.00      1.00       503



Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

### 📌 Analisis & Perbandingan
- Random Forest, sebagai kumpulan pohon keputusan memberikan hasil sempurna pada data ini. Namun, dibandingkan Decision Tree, Random Forest umumnya lebih stabil dan resisten terhadap overfitting karena menggunakan rata-rata prediksi dari banyak pohon. Maka, jika kedua model memberi hasil yang sama, Random Forest cenderung lebih dipercaya untuk generalisasi.

- Decision Tree berhasil mengklasifikasikan semua data dengan sempurna tanpa kesalahan sama sekali. Ini menunjukkan bahwa model sangat fit terhadap data pelatihan dan/atau data uji. Performa sempurna ini bisa menjadi indikasi overfitting jika diuji hanya pada data pelatihan, karena Decision Tree cenderung membangun model yang sangat kompleks mengikuti noise di data.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

Tuning Model RandomForest dan Decision Tree dengan GridSearchCV

In [18]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='f1_macro')
grid_search.fit(X_train, y_train)
best_rf = grid_search.best_estimator_
y_pred_best_rf = best_rf.predict(X_test)

In [19]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


dt = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_dt = grid_search.best_estimator_
y_pred_best_dt = best_dt.predict(X_test)

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [20]:
#hasil tuning model rf
print("Random Forest model")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_best_rf))



Random Forest model
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00       113
           2       1.00      1.00      1.00       124
           3       1.00      1.00      1.00       114
           4       1.00      1.00      1.00       152

    accuracy                           1.00       503
   macro avg       1.00      1.00      1.00       503
weighted avg       1.00      1.00      1.00       503



In [21]:
#hasil tuning model dt
print("=== Optimized Decision Tree ===")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Classification Report:\n", classification_report(y_test, y_pred_best_dt))

=== Optimized Decision Tree ===
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00       113
           2       1.00      1.00      1.00       124
           3       1.00      1.00      1.00       114
           4       1.00      1.00      1.00       152

    accuracy                           1.00       503
   macro avg       1.00      1.00      1.00       503
weighted avg       1.00      1.00      1.00       503



## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

#### Perbandingan Hasil Evaluasi
Model Decision Tree maupun Random Forest menunjukkan hasil evaluasi yang sama sebelum dan sesudah tuning, dengan akurasi, precision, recall, dan f1-score yang semuanya mencapai 1.00. Ini menandakan bahwa proses tuning tidak memberikan dampak signifikan terhadap performa model, kemungkinan karena data yang digunakan sudah sangat sesuai atau model terlalu menyesuaikan diri dengan data pelatihan.

#### Kelemahan Model
Meskipun hasil evaluasi terlihat sempurna, hal ini justru menunjukkan potensi overfitting, terutama pada model Decision Tree yang memang cenderung membentuk aturan kompleks jika tidak dibatasi. Tidak adanya perbedaan antara hasil sebelum dan sesudah tuning serta metrik evaluasi yang sangat tinggi di semua kelas bisa mengindikasikan bahwa model dilatih dan diuji pada data yang sama, atau tidak dilakukan validasi yang memadai.

#### Rekomendasi Tindakan
- Optimasi data : Perbanyak data agar model belajar lebih umum, atau gunakan teknik seperti SMOTE untuk menyeimbangkan kelas.Kurangi jumlah fitur yang tidak penting dengan PCA atau seleksi fitur agar noise berkurang.
- kurangi fitur yang tidak perlu : Jika terlalu banyak fitur, model bisa belajar dari informasi yang tidak penting. Gunakan teknik seperti seleksi fitur atau PCA untuk menyaring hanya fitur yang benar-benar membantu prediksi.
- Uji dengan data baru yang belum pernah dipelajari model : dengan Hasil evaluasi yang tinggi tidak cukup jika hanya diuji di data pelatihan. Model sebaiknya diuji pada data lain yang tidak digunakan saat melatih agar kita tahu seberapa baik model bekerja di kondisi nyata.
- Mencoba algoritma lain : Jika model saat ini masih menunjukkan tanda-tanda overfitting, seperti akurasi sempurna di data latih namun buruk di data uji, maka mencoba algoritma lain yang lebih sederhana atau menerapkan pendekatan ensemble dengan teknik regularisasi menjadi langkah penting untuk memperoleh model yang lebih general dan stabil.
